In [1]:
# Install the required libraries
# !pip install requests pandas matplotlib seaborn

# Import the libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time

In [3]:
def get_city_coords(city, api_key):
    """Gets the latitude and longitude for a city."""
    geo_url = "http://api.openweathermap.org/geo/1.0/direct"
    params = {'q': city, 'limit': 1, 'appid': api_key}
    try:
        response = requests.get(geo_url, params=params)
        response.raise_for_status()
        data = response.json()
        if data:
            return data[0]['lat'], data[0]['lon']
    except requests.exceptions.HTTPError as e:
        print(f"Error finding coordinates for {city}: {e}")
    return None, None

def get_historical_weather(lat, lon, date_timestamp, api_key):
    """Fetches historical weather for a specific lat, lon, and timestamp."""
    hist_url = "https://api.openweathermap.org/data/3.0/onecall/timemachine"
    params = {
        'lat': lat,
        'lon': lon,
        'dt': date_timestamp,
        'appid': api_key,
        'units': 'metric'
    }
    try:
        response = requests.get(hist_url, params=params)
        response.raise_for_status()
        data = response.json()
        # Calculate the average temperature from the hourly data provided
        hourly_data = data['data'][0]['hourly']
        avg_temp = sum(hour['temp'] for hour in hourly_data) / len(hourly_data)
        return avg_temp
    except requests.exceptions.HTTPError as e:
        if response.status_code == 401:
            print("Error: Invalid API key.")
        else:
            print(f"HTTP error fetching historical data: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return None

In [9]:
# --- IMPORTANT: PASTE YOUR API KEY HERE ---
API_KEY = "" 

# List of cities
cities = ["Lagos", "Tokyo", "New York", "London", "Sydney", "Dubai"]

# List to store all our data
all_weather_data = []

# Get today's date
today = datetime.utcnow().date()

print("Fetching historical weather data for the last 5 days...")

for city in cities:
    lat, lon = get_city_coords(city, API_KEY)
    
    if lat and lon:
        print(f"\nProcessing {city}...")
        for i in range(1, 6): # Loop for the last 5 days
            # Get the date for the past day
            past_date = today - timedelta(days=i)
            # Convert date to a Unix timestamp for the API
            timestamp = int(datetime(past_date.year, past_date.month, past_date.day, 12).timestamp())
            
            # Fetch the average temperature for that day
            avg_temp = get_historical_weather(lat, lon, timestamp, API_KEY)
            
            if avg_temp is not None:
                all_weather_data.append({
                    'City': city,
                    'Date': past_date,
                    'Avg Temperature (°C)': avg_temp
                })
            # Pause briefly to avoid hitting API rate limits
            time.sleep(0.5)

# Create a DataFrame
df_hist = pd.DataFrame(all_weather_data)

print("\nHistorical Weather Data:")
display(df_hist)

C:\Users\New\AppData\Local\Temp\ipykernel_2584\4284126487.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow().date()


Fetching historical weather data for the last 5 days...
Error finding coordinates for Lagos: 401 Client Error: Unauthorized for url: http://api.openweathermap.org/geo/1.0/direct?q=Lagos&limit=1&appid=6ba1814960c84a019709d249a1bb095d
Error finding coordinates for Tokyo: 401 Client Error: Unauthorized for url: http://api.openweathermap.org/geo/1.0/direct?q=Tokyo&limit=1&appid=6ba1814960c84a019709d249a1bb095d
Error finding coordinates for New York: 401 Client Error: Unauthorized for url: http://api.openweathermap.org/geo/1.0/direct?q=New+York&limit=1&appid=6ba1814960c84a019709d249a1bb095d
Error finding coordinates for London: 401 Client Error: Unauthorized for url: http://api.openweathermap.org/geo/1.0/direct?q=London&limit=1&appid=6ba1814960c84a019709d249a1bb095d
Error finding coordinates for Sydney: 401 Client Error: Unauthorized for url: http://api.openweathermap.org/geo/1.0/direct?q=Sydney&limit=1&appid=6ba1814960c84a019709d249a1bb095d
Error finding coordinates for Dubai: 401 Client E

""


In [ ]:
# Ensure the DataFrame is not empty before plotting
if not df_hist.empty:
    print("\nComparing Temperature Trends Over the Last 5 Days:")
    
    plt.figure(figsize=(14, 8))
    
    # Use seaborn's lineplot to automatically plot each city's trend
    sns.lineplot(data=df_hist, x='Date', y='Avg Temperature (°C)', hue='City', marker='o')
    
    plt.title('5-Day Temperature Trend Comparison', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Average Temperature (°C)', fontsize=12)
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.legend(title='City')
    plt.tight_layout()
    plt.show()